In [1]:
# !pip install -r requirements.txt

In [2]:
from platform import python_version

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from qdrant_client.models import VectorParams, Distance
from dotenv import load_dotenv
load_dotenv()
import os

from typing import List, Optional
import os
from pathlib import Path

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

### альтернативный вариант

In [3]:
from platform import python_version
import os
from pathlib import Path
from typing import List, Optional

from dotenv import load_dotenv
load_dotenv()

import re
import uuid
import pandas as pd

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter

import clickhouse_connect

# (если понадобится) from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader


In [4]:
# 2.1 Настройка подключения к Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
client_qdrant = QdrantClient(url=QDRANT_URL)

# 2.2 Настройка подключения к ClickHouse
CH_HOST = '84.201.160.255'   # или из окружения
CH_PORT = 8123
CLICKHOUSE_USER = 'peter'
CLICKHOUSE_PASSWORD = '1234'

client_clickhouse = clickhouse_connect.get_client(
    host=CH_HOST,
    port=CH_PORT,
    username=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD
)


In [5]:
emails_df = client_clickhouse.query_df("""
    SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
           sent_at_utc, folder, body_text, body_html
    FROM mailkb.emails
    ORDER BY sent_at_utc DESC, message_id DESC
    LIMIT 100
""")


Настройка коллекции в Qdrant (с вариантами)

In [6]:
BASE_URL = "http://localhost:8000/v1"
embeddings = OpenAIEmbeddings(
    # model="Qwen/Qwen3-Embedding-8B",
    model="Qwen/Qwen3-Embedding-0.6B",
    api_key="not-needed",
    base_url=BASE_URL,
    tiktoken_enabled=False,
)

In [7]:
# тест - получаем размерность эмбединга
vec = embeddings.embed_query("test")
EMBEDDING_DIM = len(vec)
EMBEDDING_DIM

1024

In [8]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain_qdrant import QdrantVectorStore

COLLECTION_NAME = "mailkb_emails"


# ---------- Вариант A: УДАЛИТЬ коллекцию и СОЗДАТЬ ЗАНОВО ----------
def setup_collection_recreate():
    """
    Полная пересоздача коллекции: удаляем старую, создаём новую с нужной размерностью.
    Полезно, когда меняешь модель / размер эмбеддинга.
    """
    client_qdrant.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=EMBEDDING_DIM,
            distance=Distance.COSINE,
        ),
    )

    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # если не хочешь, чтобы он дополнительно валидировал конфиг:
        # validate_collection_config=False,
    )
    return qv


# ---------- Вариант B: СОЗДАТЬ, если коллекции ещё нет ----------
def setup_collection_create_if_not_exists():
    """
    Если коллекции нет — создаём.
    Если есть — НЕ трогаем (данные сохраняются).
    """
    collections = client_qdrant.get_collections().collections
    existing_names = {c.name for c in collections}

    if COLLECTION_NAME not in existing_names:
        client_qdrant.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=EMBEDDING_DIM,
                distance=Distance.COSINE,
            ),
        )

    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # при желании можно отключить проверку:
        # validate_collection_config=False,
    )
    return qv


# ---------- Вариант C: ИСПОЛЬЗОВАТЬ ТЕКУЩУЮ коллекцию как есть ----------
def setup_collection_use_existing():
    """
    Просто подключаемся к уже существующей коллекции.
    Ничего не создаём и не удаляем.
    Важно: размерность в Qdrant должна совпадать с размерностью эмбеддинга.
    """
    qv = QdrantVectorStore(
        client=client_qdrant,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        # если вдруг Qdrant создан с "левым" размером, и ты осознанно хочешь
        # отключить проверку (НЕ рекомендую в бою):
        # validate_collection_config=False,
    )
    return qv



In [9]:
# Пример использования одного из вариантов:
# qv = setup_collection_recreate()
qv = setup_collection_create_if_not_exists()
# qv = setup_collection_use_existing()

Подготовка утилит (нормализация, участники, превращение в документы)

In [10]:
RE_PREFIX = re.compile(r'^\s*(re|fw|fwd):\s*', flags=re.IGNORECASE)

def normalize_subject(subj: str) -> str:
    s = subj or ""
    while True:
        ns = RE_PREFIX.sub('', s).strip()
        if ns == s:
            break
        s = ns
    s = re.sub(r'\s+', ' ', s)
    return s.lower()

def participants_list(row) -> list[str]:
    def _norm(x):
        if not x:
            return []
        if isinstance(x, list):
            return [str(i).strip() for i in x if str(i).strip()]
        return [p.strip() for p in re.split(r'[;,]', str(x)) if p.strip()]
    people = _norm(row.get("from_addr")) + _norm(row.get("to_addr")) \
           + _norm(row.get("cc_addr")) + _norm(row.get("bcc_addr"))
    return sorted(set(people))

def build_docs(df: pd.DataFrame) -> list[Document]:
    docs: list[Document] = []
    for _, r in df.iterrows():
        subj = (r.get("subject") or "").strip()
        body = (r.get("body_text") or "").strip()
        if not body:
            body = (r.get("body_html") or "").strip()
        text = (subj + "\n\n" + body).strip()
        if not text:
            continue
        parts = participants_list(r)
        norm_subj = normalize_subject(subj)
        thread_key = f"{norm_subj}||{';'.join(sorted(parts))}"
        meta = {
            "row_id": r.get("id"),
            "message_id": r.get("message_id"),
            "subject": subj,
            "sent_at_utc": str(r.get("sent_at_utc")),
            "folder": r.get("folder"),
            "from_addr": parts[:1],
            "participants": parts,
            "thread_key": thread_key,
        }
        docs.append(Document(page_content=text, metadata=meta))
    return docs


Очистка текста и разбивка на чанки

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)

RE_QUOTED = re.compile(r"(?m)^(>+).*$")  # строки, начинающиеся с >
RE_HDR = re.compile(r"(?:^|\n)(from:|sent:|to:|subject:).*(?:\n.*){0,20}", re.IGNORECASE)

def clean_text(t: str) -> str:
    t = t.replace("\r\n", "\n")
    t = RE_HDR.sub("\n", t)
    t = RE_QUOTED.sub("", t)
    t = "\n".join([ln.strip() for ln in t.split("\n") if ln.strip()])
    return t

def preprocess_and_chunk(docs: list[Document]) -> list[Document]:
    cleaned: list[Document] = []
    for d in docs:
        txt = clean_text(d.page_content)
        if not txt:
            continue
        cleaned.append(Document(page_content=txt, metadata=d.metadata))
    return splitter.split_documents(cleaned)


Генерация стабильных ID и загрузка в Qdrant

In [12]:
def make_ids(chunks: list[Document]) -> list[str]:
    counters: dict[str, int] = {}
    ids: list[str] = []
    for c in chunks:
        msg = c.metadata.get("message_id") or c.metadata.get("row_id") or "noid"
        i = counters.get(msg, 0)
        counters[msg] = i + 1
        raw = f"{msg}::chunk_{i}"
        uid = uuid.uuid5(uuid.NAMESPACE_URL, raw)
        ids.append(str(uid))
    return ids

def process_batch(df: pd.DataFrame):
    docs = build_docs(df)
    chunks = preprocess_and_chunk(docs)
    if not chunks:
        return
    ids = make_ids(chunks)
    texts = [c.page_content for c in chunks]
    metadatas = [c.metadata for c in chunks]
    qv.add_texts(texts=texts, metadatas=metadatas, ids=ids)


Запуск загрузки (батчинг)

In [13]:
# Пример: полный цикл с батчингом
# BATCH_SIZE = 5000
# offset = 0
#
# while True:
#     df_batch = client_clickhouse.query_df(f"""
#         SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
#                sent_at_utc, folder, body_text, body_html
#         FROM mailkb.emails
#         ORDER BY sent_at_utc ASC
#         LIMIT {BATCH_SIZE} OFFSET {offset}
#     """)
#     if df_batch.empty:
#         break
#     process_batch(df_batch)
#     offset += len(df_batch)
#     print(f"Processed {offset} rows")

# Или — одноразовая загрузка небольшой выборки
offset = 0
df_small = client_clickhouse.query_df("""
    SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr,
           sent_at_utc, folder, body_text, body_html
    FROM mailkb.emails
    ORDER BY sent_at_utc DESC, message_id DESC
    LIMIT 100
""")
process_batch(df_small)
offset += len(df_small)


Аналитика переписки (треды, темы, теги проектов)

In [14]:
from collections import Counter
# Утилиты
def split_addrs(x) -> list[str]:
    if x is None:
        return []
    if isinstance(x, list):
        return [a.strip() for a in x if str(a).strip()]
    return [a.strip() for a in re.split(r'[;,]', str(x)) if a.strip()]

def participants(row) -> list[str]:
    lst = split_addrs(row.get("from_addr")) + split_addrs(row.get("to_addr")) \
        + split_addrs(row.get("cc_addr")) + split_addrs(row.get("bcc_addr"))
    return sorted(set(lst))

def clean_for_topics(text: str) -> list[str]:
    if not text:
        return []
    t = text.lower()
    t = re.sub(r'[^a-zа-я0-9\s\-]+', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    toks = t.split()
    stop = set("""
        и в во на с со от до по за для при как что это это/that of the a an to is are was were be been being
        у о об обo про из из-за над под между но или либо либо/или который которые какая какие чей чья чей-то
        re fw fwd subject тема письмо письма письме по- поводу
    """.split())
    return [w for w in toks if len(w) > 2 and w not in stop]

PROJECT_KEYWORDS = {
    "segezha": "Проект: Segezha",
    "ewm": "SAP EWM",
    "bw": "SAP BW",
    "o2c": "O2C",
    "mnf": "MNF",
    "сцбк": "СЦБК",
    "вагон": "Логистика/Вагоны",
}

def guess_project_tags(subject: str, body: str, addrs: list[str]) -> list[str]:
    text = f"{subject or ''} {body or ''}".lower()
    tags = set()
    for kw, tag in PROJECT_KEYWORDS.items():
        if kw in text:
            tags.add(tag)
    for a in addrs:
        m = re.search(r'@([a-z0-9\.-]+)', a.lower())
        if m:
            dom = m.group(1)
            if 'segezha' in dom:
                tags.add("Проект: Segezha")
            if 'bearingpoint' in dom:
                tags.add("Внутренние/Подрядчик")
    return sorted(tags)

# Добавление колонок в df
df = emails_df.copy()
df["norm_subject"] = df["subject"].apply(lambda s: normalize_subject(s if isinstance(s, str) else ""))
df["participants"] = df.apply(participants, axis=1)
df["thread_key"] = df.apply(lambda r: f'{r["norm_subject"]}||{";".join(r["participants"])}', axis=1)

def plain_body(row):
    t = (row.get("body_text") or "").strip()
    if t:
        return t
    return (row.get("body_html") or "").strip()

df["plain_body"] = df.apply(plain_body, axis=1)
df["topics"] = df.apply(lambda r: clean_for_topics(r["subject"]) + clean_for_topics(r["plain_body"][:1000]), axis=1)
df["project_tags"] = df.apply(lambda r: guess_project_tags(r["subject"], r["plain_body"], r["participants"]), axis=1)

threads = (
    df.groupby("thread_key")
      .agg(
         first_sent=("sent_at_utc","min"),
         last_sent=("sent_at_utc","max"),
         n_emails=("id","count"),
         subjects=("subject", lambda s: list(pd.unique([x for x in s if isinstance(x, str)]))[:5]),
         participants=("participants", lambda cols: sorted(set(sum(cols, [])))),
         projects=("project_tags", lambda cols: sorted(set(sum(cols, [])))),
         topics=("topics", lambda cols: [w for w,_ in Counter(sum(cols, [])).most_common(10)])
      )
      .reset_index()
      .sort_values(["last_sent","n_emails"], ascending=[False, False])
)

def flatten(col):
    out = []
    for arr in col:
        out.extend(arr)
    return out

projects_df = (
    threads.assign(project=lambda x: x["projects"].apply(lambda arr: arr if arr else ["(Не классифицировано)"]))
           .explode("project")
           .groupby("project")
           .agg(
              n_threads=("thread_key","nunique"),
              n_emails=("n_emails","sum"),
              first_sent=("first_sent","min"),
              last_sent=("last_sent","max"),
              participants=("participants", lambda cols: sorted(set(flatten(cols)))[:50]),
              top_topics=("topics", lambda cols: [w for w,_ in Counter(flatten(cols)).most_common(15)])
           )
           .reset_index()
           .sort_values(["n_threads","n_emails"], ascending=[False, False])
)


C:\Users\Peter\AppData\Local\Temp\ipykernel_158240\3129374694.py:77: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  subjects=("subject", lambda s: list(pd.unique([x for x in s if isinstance(x, str)]))[:5]),


Сводка тредов с помощью LLM

In [15]:
import json
from collections import defaultdict
from langchain.tools import tool


@tool
def search_project_emails(project_hint: str, limit: int = 200) -> str:
    """
    Найти и сгруппировать письма по проекту/теме.

    Вход:
        project_hint: текстовая подсказка — название проекта, код ЗНИ, ключевые слова.
        limit: максимум писем для выборки.

    Логика:
      1) делаем семантический поиск по Qdrant;
      2) группируем результаты по thread_key;
      3) сортируем письма внутри треда по sent_at_utc.

    Выход:
        JSON-строка со структурой:
        {
          "project_hint": "...",
          "threads": [
            {
              "thread_key": "...",
              "subject": "...",
              "participants": [...],
              "messages": [
                {
                  "message_id": "...",
                  "sent_at_utc": "...",
                  "from_addr": "...",
                  "snippet": "...",
                  "subject": "...",
                  "folder": "...",
                  "participants": [...],
                  "metadata": {...}
                }
              ]
            }
          ]
        }
    """
    # 1. Семантический поиск по тексту
    docs = qv.similarity_search(project_hint, k=limit)

    threads = defaultdict(list)

    for doc in docs:
        md = doc.metadata or {}

        thread_key = md.get("thread_key") or md.get("subject") or "unknown_thread"

        message = {
            "row_id": md.get("row_id"),
            "message_id": md.get("message_id"),
            "sent_at_utc": md.get("sent_at_utc"),
            "from_addr": (md.get("from_addr") or [None])[0],
            "subject": md.get("subject"),
            "folder": md.get("folder"),
            "participants": md.get("participants") or [],
            # обрезаем тело, чтобы не заливать в ответ мегатекст
            "snippet": (doc.page_content[:600] + "…") if doc.page_content else None,
            "metadata": md,
        }

        threads[thread_key].append(message)

    thread_list = []
    for thread_key, messages in threads.items():
        # сортируем по дате
        messages_sorted = sorted(
            messages,
            key=lambda m: m.get("sent_at_utc") or "",
        )

        # subject — из какого-то письма (обычно это RE:/FW: и т.п.)
        subject = None
        for m in messages_sorted:
            if m.get("subject"):
                subject = m["subject"]
                break

        # участники: агрегируем по всем письмам
        participants_set = set()
        for m in messages_sorted:
            for p in m.get("participants") or []:
                participants_set.add(p)

        thread_list.append(
            {
                "thread_key": thread_key,
                "subject": subject,
                "participants": sorted(participants_set),
                "messages": messages_sorted,
            }
        )

    result = {
        "project_hint": project_hint,
        "threads": thread_list,
    }

    return json.dumps(result, ensure_ascii=False, indent=2)


In [16]:
@tool
def search_emails_raw(query: str, limit: int = 50) -> str:
    """
    Общий поиск писем в Qdrant по любому запросу (без группировки).
    Удобно для отладки.
    """
    docs = qv.similarity_search(query, k=limit)
    data = []
    for doc in docs:
        md = doc.metadata or {}
        data.append({
            "thread_key": md.get("thread_key"),
            "row_id": md.get("row_id"),
            "message_id": md.get("message_id"),
            "subject": md.get("subject"),
            "sent_at_utc": md.get("sent_at_utc"),
            "from_addr": (md.get("from_addr") or [None])[0],
            "participants": md.get("participants") or [],
            "folder": md.get("folder"),
            "snippet": (doc.page_content[:400] + "…") if doc.page_content else None,
        })
    return json.dumps({"query": query, "results": data}, ensure_ascii=False, indent=2)

In [17]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
    PIIMiddleware,
    SummarizationMiddleware,
)
# from langchain_tavily import TavilySearch  # если понадобится веб-поиск

SYSTEM_PROMPT = """
Ты — аналитик переписки по проектам.

Твоя задача: по запросу пользователя сделать отчёт по проекту,
используя письма из базы (Qdrant).

Алгоритм действий:
1) Понять по формулировке пользователя, какой проект/тема интересует
   (например, компания, ЗНИ, код проекта, ключевые слова).
2) Вызвать инструмент `search_project_emails` с осмысленным project_hint:
   - в hint можно использовать название проекта,
     ключевые слова (например, "Segezha", "ЗНИ 239", "номер вагона" и т.п.).
3) Из результата инструмента:
   - просмотреть список `threads`;
   - для каждого треда понять:
       * о чём тред (по subject, snippet'ам писем),
       * какие решения были приняты,
       * какие действия согласованы,
       * есть ли открытые вопросы/риски,
       * кто основные участники.
4) Сформировать отчёт для пользователя:

Формат ответа:
1. Краткое резюме по проекту (2–10 абзацев):
   - основная тема обсуждений,
   - ключевые решения и договорённости,
   - текущий статус (по возможности).
2. Таблица или маркированный список по тредам:
   - Название/subject треда,
   - основная тема,
   - краткое содержание (1–3 пункта),
   - итог/статус.
3. Если информации недостаточно или она шумная — обязательно упомяни это.
"""

agent = create_agent(
    model="gpt-5",  # или твой модельный endpoint
    tools=[
        search_project_emails,
        search_emails_raw,   # опционально
    ],
    system_prompt=SYSTEM_PROMPT,
    middleware=[
        # скрываем e-mail и телефоны в вводе/выводе (настраиваем по желанию)
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware(
            "phone_number",
            detector=(
                r"(?:\+?\d{1,3}[\s.-]?)?"
                r"(?:\(?\d{2,4}\)?[\s.-]?)?"
                r"\d{3,4}[\s.-]?\d{4}"
            ),
            strategy="redact",
        ),
        # если переписка большая, middleware будет автоматически её подрезать и резюмировать
        SummarizationMiddleware(
            model="gpt-5",
            max_tokens_before_summary=500,
        ),
    ],
)


In [18]:
result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": """Сделай отчёт по проекту Segezha на основе переписки. Какие темы поднимались в переписках, для каждой темы краткое резюме к чему пришли в итоге

            Если идет переписка по локальному вопросу, например по конкретному Бизнес-партнеру, конкретному материалу, что какой-то документ не выгрузился, или какой-то конкретный документ надо скорректировать и т.д., то нужно очень верхнеуровнево одной строкой отметить суть, не перечисляя каждый такой инцидент. Если таких локальных вопросов повторяется определенное количество, то стоит отметить повторяемость таких вопросов/инцидентов """

        }
    ]
})

report = result["messages"][-1].content
print(report)


Отчет по проекту Segezha (по переписке)

Краткое резюме
- Основные темы периода: сверка и исправление внутригрупповых оборотов (ВГО) и валютных расхождений; интеграция TM–SD для корректного расчета выручки (ЗНИ 272); отражение и распределение фактических транспортных затрат и начислений; утверждение регламентов по TM; ряд эксплуатационных инцидентов (интерфейсы с Transporeon, НСИ/БП, корректировки печатных форм и ролей).
- ВГО/валюты: согласован порядок работы с расхождениями, подготовлено и согласовано ЗНИ 281; определены сроки корректировок (при согласовании – до 5 марта). По «Зеркалу 2» внесены правки (двойное налогообложение, кодировки валют), но остались вопросы по кодам валют (EUR vs UEU) и работе зеркала при нескольких ДРФ в одном документе.
- TM–SD (ЗНИ 272): согласовано решение по передаче даты коносамента из TM и заполнению даты перехода рисков (FOB/CFR/CIF) в исходящей поставке. Заказчик логистика, требуется расширение объема ЗНИ (контроли в SD); ориентировочный срок продукт

### вывод инфо